In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os

In [9]:
# Unzipping the file
with zipfile.ZipFile('Students_Performance.zip', 'r') as zip_ref:
    zip_ref.extractall('Students_Performance')

# Get list of all files in the extracted folder
file_list = os.listdir('Students_Performance')

# Load the first CSV file into a DataFrame as an example
csv_file = [file for file in file_list if file.endswith('.csv')][0]
df = pd.read_csv(f'Students_Performance/{csv_file}')

   gender race_ethnicity parental_level_of_education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test_preparation_course  math_score  reading_score  writing_score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


### Initial Data Exploration

In [11]:
# Display the first few rows to inspect the data
print(df.head())

   gender race_ethnicity parental_level_of_education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test_preparation_course  math_score  reading_score  writing_score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


In [13]:
df.describe()

,math_score,reading_score,writing_score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race_ethnicity               1000 non-null   object
 2   parental_level_of_education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test_preparation_course      1000 non-null   object
 5   math_score                   1000 non-null   int64 
 6   reading_score                1000 non-null   int64 
 7   writing_score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [19]:
# Display unique values in categorical columns
categorical_columns = df.select_dtypes(include='object').columns
for col in categorical_columns:
    print(f"\nUnique values in '{col}':")
    print(df[col].unique())


Unique values in 'gender':
['female' 'male']

Unique values in 'race_ethnicity':
['group B' 'group C' 'group A' 'group D' 'group E']

Unique values in 'parental_level_of_education':
["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']

Unique values in 'lunch':
['standard' 'free/reduced']

Unique values in 'test_preparation_course':
['none' 'completed']


### Handling missing values

In [22]:
# Fill missing numerical values with the median
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Fill missing categorical values with the most frequent value (mode)
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Verify if all missing values have been handled
print("\nMissing Values After Handling:\n")
print(df.isnull().sum())


Missing Values After Handling:

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64


In [ ]:
# df.fillna(method='ffill', inplace=True)
# df.fillna(method='bfill', inplace=True)

In [ ]:
# df.isnull().sum()

### Smoothing Noisy Data

In [30]:
# Apply rolling mean to smooth the 'math_score' column
df['math_score'] = df['math_score'].rolling(window=3, min_periods=1).mean().fillna(df['math_score'])

# Similarly, smooth 'reading_score' and 'writing_score'
df['reading_score'] = df['reading_score'].rolling(window=3, min_periods=1).mean().fillna(df['reading_score'])
df['writing_score'] = df['writing_score'].rolling(window=3, min_periods=1).mean().fillna(df['writing_score'])

print(df.head())

   gender race_ethnicity parental_level_of_education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test_preparation_course  math_score  reading_score  writing_score  
0                    none   72.000000      72.000000      74.000000  
1               completed   70.500000      81.000000      81.000000  
2                    none   77.000000      85.666667      85.000000  
3                    none   68.666667      80.666667      75.000000  
4                    none   71.000000      76.666667      70.666667  


### Identifying and Removing Outliers

In [40]:
# Identify outliers using IQR method for numeric columns
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers from all numeric columns
for col in numeric_cols:
    df = remove_outliers(df, col)

print("\nData Shape After Removing Outliers:", df.shape)



Data Shape After Removing Outliers: (978, 8)


### Verify Data Cleaning

In [44]:
# Check for missing values again
print(df.isnull().sum())

# Get descriptive statistics again
print(df.describe())

# Display the first few rows of the cleaned dataset
print(df.head())

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64
       math_score  reading_score  writing_score
count  978.000000     978.000000     978.000000
mean    66.491650      69.550102      68.459100
std      8.056801       7.927228       8.248788
min     45.000000      48.333333      46.333333
25%     61.000000      64.333333      63.000000
50%     66.500000      69.666667      68.666667
75%     72.000000      75.333333      74.333333
max     87.666667      91.000000      91.000000
   gender race_ethnicity parental_level_of_education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male  

### Final check

In [46]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 978 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       978 non-null    object 
 1   race_ethnicity               978 non-null    object 
 2   parental_level_of_education  978 non-null    object 
 3   lunch                        978 non-null    object 
 4   test_preparation_course      978 non-null    object 
 5   math_score                   978 non-null    float64
 6   reading_score                978 non-null    float64
 7   writing_score                978 non-null    float64
dtypes: float64(3), object(5)
memory usage: 68.8+ KB


,math_score,reading_score,writing_score
count,978.000000,978.000000,978.000000
mean,66.491650,69.550102,68.459100
std,8.056801,7.927228,8.248788
min,45.000000,48.333333,46.333333
25%,61.000000,64.333333,63.000000
50%,66.500000,69.666667,68.666667
75%,72.000000,75.333333,74.333333
max,87.666667,91.000000,91.000000


In [ ]:
'''
Exploratory Data Analysis (EDA):

    Plot distributions of scores.
    Examine relationships between different features.

Feature Engineering:

    Create new features if necessary.
    Encode categorical variables.

Modeling:

    Train machine learning models on the cleaned data.
    Evaluate model performance.
'''